In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from hmmlearn import hmm
import string
from gensim import corpora
from gensim.utils import simple_preprocess
from time import time
from joblib import Parallel, delayed
from collections import Counter

from utils import prep_text, prep_data, vec_translate

# Speaker identification

In [2]:
# read in all talks for the 20 most frequent speakers
n_speakers = 20
summary = pd.read_json("../merged_summary_topics.json")
top_speakers = summary["Speaker"].value_counts()[:n_speakers].index.to_list()

talks = {}
for name in top_speakers:
    talks[name] = []
    for filename in summary[summary["Speaker"] == name]["File"]:
        with open("../" + filename, "r") as f:
            text = f.read()
            processed = simple_preprocess(text)
            if len(text):
                talks[name].append(processed)

In [10]:
# concatenate the first 10 of President Monson's talks
name = 'Thomas S. Monson'
text = sum(talks[name][:10], start=[])

# for training on the vocabulary of every talk in the dataset
"""corpus = sum(talks.values(), start=[])
dictionary = corpora.Dictionary(corpus)"""

# I was getting errors when training on the entire vocabulary (maybe it's
# too big?) so I switched to training on just 10 of Monson's talks
dictionary = corpora.Dictionary([text])

In [11]:
model = hmm.MultinomialHMM(n_components=5, n_iter=100)
model.fit(prep_text(text, dictionary))

MultinomialHMM(n_components=5, n_iter=100,
               random_state=RandomState(MT19937) at 0x7FE364551740)

In [12]:
# find the talk with the highest log probability
print("Score for a talk from Monson in the training data:",
     model.score(prep_text(talks[name][0], dictionary))
)
print("Score for a talk from Monson not in training data:",
      model.score(prep_text(talks[name][11], dictionary)))

max_score, max_name = -np.inf, None
for name in list(talks.keys()):
    score = model.score(prep_text(talks[name][0], dictionary))
    print(name, ":", score)
    
    if score > max_score:
        max_score, max_name = score, name

print(
    f"\nSpeaker with the maximum score: {max_name} with score = {max_score}"
)

Score for a talk from Monson in the training data: -12428.838065393451
Score for a talk from Monson not in training data: -19814.614154689396
Thomas S. Monson : -12428.838065393451
Gordon B. Hinckley : -19542.79811896064
James E. Faust : -6044.043404057589
Boyd K. Packer : -13173.54626312474
L. Tom Perry : -1671.975462819844
Henry B. Eyring : -13349.163098859106
M. Russell Ballard : -3148.2043479518215
Russell M. Nelson : -inf
Dallin H. Oaks : -4915.360349456056
Spencer W. Kimball : -28470.24038003811
Ezra Taft Benson : -21211.719588135784
Richard G. Scott : -3772.9260529611183
Dieter F. Uchtdorf : -9729.56067646322
David B. Haight : -16613.942504813876
Robert D. Hales : -10272.435007644102
Marion G. Romney : -19821.63431490711
Joseph B. Wirthlin : -6244.144251418623
Jeffrey R. Holland : -19461.97952079512
Howard W. Hunter : -15937.20690994325
Neal A. Maxwell : -inf

Speaker with the maximum score: L. Tom Perry with score = -1671.975462819844


# Text generation

In [6]:
# using the previously trained model, sample 100 words
" ".join([dictionary[i] for i in model.sample(100)[0].flatten()])

'be the work heard saints the frequently to nominate had spacious meetings in driving but you earth of the inward of for the he of philippines to of my point generation the eyes in truth the to will fame for it new the way of with you of in day of fame scriptures of with better of wonder the moment form neglected morn for and mother returning not the hour armentieres him receive lord can joseph daily an god we ago paul treat day parents wanted instant thee but be don and which and so character jesus of me was'

# Character-level text generation

In [ ]:
symbols, obs = prep_data("../data/2000.txt")

model = hmm.MultinomialHMM(n_components=50, n_iter=20)
model.fit(obs.reshape(-1, 1))
X, _ = model.sample(100)
X = X.flatten()
"".join([symbols[i] for i in X])

# Speaker identification with multiple models

In [38]:
def train_model(name):
    text = sum(talks[name][:10], start=[])
    dictionary = corpora.Dictionary([text])
    
    model = hmm.MultinomialHMM(n_components=10, n_iter=100)
    model.fit(prep_text(text, dictionary))
    return model, dictionary

In [39]:
# train 20 models, one for each speaker
speakers = list(talks.keys())
models = Parallel(n_jobs=-1, verbose=20)(
    delayed(train_model)(s) for s in speakers
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   7 out of  20 | elapsed:  1.8min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done   9 out of  20 | elapsed:  2.7min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  3.1min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  13 out of  20 | elapsed:  3.2min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  15 out of  20 | elapsed:  3.4min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  3.8min remaining:   40.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  4.3min finished


In [41]:
results = {speaker: Counter() for speaker in speakers}

for sample_name in speakers:
    
    match_counts = results[sample_name]
    
    for sample_talk in talks[sample_name][11:]:
        max_score = -np.inf
        closest_speaker = None
        for name, (model, dictionary) in zip(speakers, models):
            score = model.score(prep_text(sample_talk, dictionary))
            if score > max_score:
                max_score = score
                closest_speaker = name

        match_counts[closest_speaker] += 1
        
    print(f"{sample_name}: {match_counts}")

Thomas S. Monson: Counter({'Henry B. Eyring': 158, 'Neal A. Maxwell': 19, 'Howard W. Hunter': 9, 'Dieter F. Uchtdorf': 3, 'Marion G. Romney': 3, 'M. Russell Ballard': 2, 'Robert D. Hales': 1, 'Boyd K. Packer': 1})
Gordon B. Hinckley: Counter({'Henry B. Eyring': 162, 'Howard W. Hunter': 15, 'Neal A. Maxwell': 10, 'Dieter F. Uchtdorf': 4, 'Marion G. Romney': 2, 'Robert D. Hales': 1, 'Dallin H. Oaks': 1, 'Spencer W. Kimball': 1})
James E. Faust: Counter({'Henry B. Eyring': 76, 'Neal A. Maxwell': 6, 'Howard W. Hunter': 4})
Boyd K. Packer: Counter({'Henry B. Eyring': 68, 'Neal A. Maxwell': 3, 'Howard W. Hunter': 2, 'Marion G. Romney': 2})
L. Tom Perry: Counter({'Henry B. Eyring': 71, 'Howard W. Hunter': 3})
Henry B. Eyring: Counter({'Henry B. Eyring': 69})
M. Russell Ballard: Counter({'Henry B. Eyring': 62, 'Neal A. Maxwell': 2, 'Howard W. Hunter': 1})
Russell M. Nelson: Counter({'Henry B. Eyring': 53, 'Howard W. Hunter': 2, 'Neal A. Maxwell': 2})
Dallin H. Oaks: Counter({'Henry B. Eyring':

In [37]:
# exclude talks from President Uchtdorf

results = {speaker: Counter() for speaker in speakers}

for sample_name in speakers:
    
    match_counts = results[sample_name]
    
    for sample_talk in talks[sample_name][11:]:
        max_score = -np.inf
        closest_speaker = None
        for name, (model, dictionary) in zip(speakers, models):
            score = model.score(prep_text(sample_talk, dictionary))
            if score > max_score and name != 'Dieter F. Uchtdorf':
                max_score = score
                closest_speaker = name

        match_counts[closest_speaker] += 1
        
    print(f"{sample_name}: {match_counts}")

Thomas S. Monson: Counter({'Dallin H. Oaks': 88, 'Neal A. Maxwell': 49, 'Marion G. Romney': 45, 'M. Russell Ballard': 5, 'Joseph B. Wirthlin': 4, 'Thomas S. Monson': 3, 'Spencer W. Kimball': 1, 'Henry B. Eyring': 1})
Gordon B. Hinckley: Counter({'Dallin H. Oaks': 94, 'Marion G. Romney': 68, 'Neal A. Maxwell': 16, 'Henry B. Eyring': 6, 'Joseph B. Wirthlin': 5, 'Spencer W. Kimball': 3, 'M. Russell Ballard': 2, 'Gordon B. Hinckley': 2})
James E. Faust: Counter({'Dallin H. Oaks': 52, 'Marion G. Romney': 24, 'Neal A. Maxwell': 9, 'M. Russell Ballard': 1})
Boyd K. Packer: Counter({'Dallin H. Oaks': 35, 'Marion G. Romney': 31, 'Neal A. Maxwell': 5, 'M. Russell Ballard': 2, 'Spencer W. Kimball': 2})
L. Tom Perry: Counter({'Dallin H. Oaks': 46, 'Marion G. Romney': 24, 'Spencer W. Kimball': 1, 'Henry B. Eyring': 1, 'M. Russell Ballard': 1, 'Joseph B. Wirthlin': 1})
Henry B. Eyring: Counter({'Henry B. Eyring': 51, 'Dallin H. Oaks': 11, 'Marion G. Romney': 6, 'Spencer W. Kimball': 1})
M. Russell B